# SORA with Azure AI Foundry
> https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/video-generation

In [1]:
import base64
import datetime
import os
import requests
import sys
import time

from dotenv import load_dotenv
from IPython.display import Video, HTML
from moviepy import *

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 30-May-2025 14:22:23


In [4]:
SORA_DIR = "videos"

os.makedirs(SORA_DIR, exist_ok=True)

In [5]:
load_dotenv("azure.env")

endpoint = os.environ['AZURE_OPENAI_ENDPOINT']
api_key = os.environ['AZURE_OPENAI_API_KEY']

model = "sora"

# Function

In [6]:
def sora(prompt, width=480, height=480, n_seconds=5, n_variants=1):
    """
    Generates a video based on the given prompt using the SORA model.

    Parameters:
    prompt (str): The text prompt to generate the video.
    width (int): The width of the video. Supported values are 480, 854, 720, 1080, and 1920.
    height (int): The height of the video. Supported values are 480, 854, 720, 1080, and 1920.
    n_seconds (int): The duration of the video in seconds. Must be between 1 and 20 seconds.
    n_variants (int): The number of video variants to generate.
    
    Returns:
    str: The filename of the generated video.

    Raises:
    Exception: If the video generation job fails or no generations are found.
    """
    start = time.time()

    api_version = 'preview'
    headers = {"api-key": api_key, "Content-Type": "application/json"}

    idx = datetime.datetime.today().strftime('%d%b%Y_%H%M%S')
    output_filename = os.path.join(SORA_DIR, f"sora_{idx}.mp4")

    # 1. Create a video generation job
    create_url = f"{endpoint}/openai/v1/video/generations/jobs?api-version={api_version}"
    body = {
        "prompt": prompt,
        "width": width,  # 480x480, 480x854, 854x480, 720x720, 720x1280, 1280x720, 1080x1080, 1080x1920, 1920x1080.
        "height": height,  # 480x480, 480x854, 854x480, 720x720, 720x1280, 1280x720, 1080x1080, 1080x1920, 1920x1080.
        "n_seconds": n_seconds,  # between 1 and 20 seconds
        "n_variants": n_variants,
        "model": model,  # SORA model
    }
    response = requests.post(create_url, headers=headers, json=body)
    response.raise_for_status()

    now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
    print(f"{now} Full response JSON:", response.json())
    print()
    
    job_id = response.json()["id"]
    now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
    print(f"{now} Job created: {job_id}")

    # 2. Poll for job status
    status_url = f"{endpoint}/openai/v1/video/generations/jobs/{job_id}?api-version={api_version}"
    status = None

    while status not in ("succeeded", "failed", "cancelled"):
        time.sleep(5)  # Wait before polling again
        status_response = requests.get(status_url, headers=headers).json()
        status = status_response.get("status")
        now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
        print(f"{now} Job status: {status}")

    # 3. Retrieve generated video
    if status == "succeeded":
        generations = status_response.get("generations", [])
        
        if generations:
            now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
            print(f"\n{now} ✅ Done. Video generation succeeded.")
            generation_id = generations[0].get("id")
            video_url = f"{endpoint}/openai/v1/video/generations/{generation_id}/content/video?api-version={api_version}"
            video_response = requests.get(video_url, headers=headers)
            
            if video_response.ok:
                # Downloading the video
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                    print(f"\nSORA Generated video saved: '{output_filename}'")

                elapsed = time.time() - start
                minutes, seconds = divmod(elapsed, 60)
                print(f"Done in {minutes:.0f} minutes and {seconds:.0f} seconds")
                
                return output_filename
        else:
            raise Exception("Error. No generations found in job result.")
    else:
        raise Exception(f"Error. Job did not succeed. Status: {status}")

## Examples

In [7]:
prompt = "Young boy and his father playing together in the ocean on the beach"

generated_video = sora(prompt, width=480, height=480, n_seconds=10, n_variants=1)

30-May-2025 14:22:26 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwgs57tyegk8s2648ywpjfcs', 'status': 'queued', 'created_at': 1748614946, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Young boy and his father playing together in the ocean on the beach', 'model': 'sora', 'n_variants': 1, 'n_seconds': 10, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 14:22:26 Job created: task_01jwgs57tyegk8s2648ywpjfcs
30-May-2025 14:22:32 Job status: queued
30-May-2025 14:22:37 Job status: running
30-May-2025 14:22:42 Job status: processing
30-May-2025 14:22:48 Job status: processing
30-May-2025 14:22:53 Job status: succeeded

30-May-2025 14:22:53 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_30May2025_142226.mp4'
Done in 0 minutes and 32 seconds


In [9]:
HTML(f"""
<video controls>
  <source src="{generated_video}" type="video/mp4">
</video>
""")

In [10]:
prompt = "An image of a realistic cloud that spells 'SORA with birds around'"

generated_video = sora(prompt, width=480, height=480, n_seconds=5, n_variants=1)

30-May-2025 14:24:31 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwgs91njfb284aebdzabbfyd', 'status': 'queued', 'created_at': 1748615071, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': "An image of a realistic cloud that spells 'SORA with birds around'", 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 14:24:31 Job created: task_01jwgs91njfb284aebdzabbfyd
30-May-2025 14:24:36 Job status: running
30-May-2025 14:24:42 Job status: running
30-May-2025 14:24:47 Job status: succeeded

30-May-2025 14:24:47 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_30May2025_142430.mp4'
Done in 0 minutes and 19 seconds


In [11]:
HTML(f"""
<video controls>
  <source src="{generated_video}" type="video/mp4">
</video>
""")

In [12]:
prompt = "Several giant wooly mammoths approach treading through a snowy meadow, their long wooly fur lightly blows in the wind as they walk, snow covered trees and dramatic snow capped mountains in the distance, mid afternoon light with wispy clouds and a sun high in the distance creates a warm glow, the low camera view is stunning capturing the large furry mammal with beautiful photography, depth of field."

generated_video = sora(prompt, width=480, height=480, n_seconds=10, n_variants=1)

30-May-2025 14:24:50 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwgs9kz5fph9z9mggdzav227', 'status': 'queued', 'created_at': 1748615090, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Several giant wooly mammoths approach treading through a snowy meadow, their long wooly fur lightly blows in the wind as they walk, snow covered trees and dramatic snow capped mountains in the distance, mid afternoon light with wispy clouds and a sun high in the distance creates a warm glow, the low camera view is stunning capturing the large furry mammal with beautiful photography, depth of field.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 10, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 14:24:50 Job created: task_01jwgs9kz5fph9z9mggdzav227
30-May-2025 14:24:55 Job status: preprocessing
30-May-2025 14:25:01 Job status: running
30-May-2025 14:25:06 Job status: running
30-May-2025 14:25:12 Job status: processing
30-May-2025 14:25:

In [13]:
HTML(f"""
<video controls>
  <source src="{generated_video}" type="video/mp4">
</video>
""")

In [14]:
prompt = "A instructional cooking session for homemade gnocchi hosted by a grandmother social media influencer set in a rustic Tuscan country kitchen with cinematic lighting."

generated_video = sora(prompt, width=480, height=480, n_seconds=5, n_variants=1)

30-May-2025 14:25:21 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwgsajy3e6nbrc6fb9f68qtv', 'status': 'queued', 'created_at': 1748615121, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'A instructional cooking session for homemade gnocchi hosted by a grandmother social media influencer set in a rustic Tuscan country kitchen with cinematic lighting.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 14:25:21 Job created: task_01jwgsajy3e6nbrc6fb9f68qtv
30-May-2025 14:25:27 Job status: queued
30-May-2025 14:25:37 Job status: succeeded

30-May-2025 14:25:37 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos/sora_30May2025_142521.mp4'
Done in 0 minutes and 18 seconds


In [15]:
HTML(f"""
<video controls>
  <source src="{generated_video}" type="video/mp4">
</video>
""")

In [16]:
prompt = "A futuristic soldier stands on the deck of a massive starship, gazing out at a galaxy filled with colorful nebulae and distant stars. She wears sleek metallic armor with glowing blue accents. In her hand, she holds a plasma rifle. The background features a bustling crew and holographic displays. The atmosphere is tense yet awe-inspiring, capturing the grandeur of space exploration. Shot in high contrast, with vivid colors and a wide-angle lens."

generated_video = sora(prompt, width=480, height=480, n_seconds=15, n_variants=1)

30-May-2025 14:25:40 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwgsb4spfez8zbbw7fzpkrrr', 'status': 'queued', 'created_at': 1748615140, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'A futuristic soldier stands on the deck of a massive starship, gazing out at a galaxy filled with colorful nebulae and distant stars. She wears sleek metallic armor with glowing blue accents. In her hand, she holds a plasma rifle. The background features a bustling crew and holographic displays. The atmosphere is tense yet awe-inspiring, capturing the grandeur of space exploration. Shot in high contrast, with vivid colors and a wide-angle lens.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 15, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 14:25:40 Job created: task_01jwgsb4spfez8zbbw7fzpkrrr
30-May-2025 14:25:45 Job status: preprocessing
30-May-2025 14:25:51 Job status: preprocessing
30-May-2025 14:25:56 Job status: running
30-May-20

In [17]:
HTML(f"""
<video controls>
  <source src="{generated_video}" type="video/mp4">
</video>
""")

In [18]:
prompt = "A young wizard with a pointy hat and a glowing wand stands in a magical forest. His robes are embroidered with golden stars, and his round glasses reflect the soft glow of fireflies. In the background, a sparkling waterfall cascades into a crystal-clear pond surrounded by glowing mushrooms. The atmosphere is enchanting and playful, evoking a sense of wonder. Shot in warm, vibrant tones, with a shallow depth of field."

generated_video = sora(prompt, width=480, height=480, n_seconds=5, n_variants=1)

30-May-2025 14:26:18 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwgsc9v5ex5vx8jf4fb5bnge', 'status': 'queued', 'created_at': 1748615178, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'A young wizard with a pointy hat and a glowing wand stands in a magical forest. His robes are embroidered with golden stars, and his round glasses reflect the soft glow of fireflies. In the background, a sparkling waterfall cascades into a crystal-clear pond surrounded by glowing mushrooms. The atmosphere is enchanting and playful, evoking a sense of wonder. Shot in warm, vibrant tones, with a shallow depth of field.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 14:26:18 Job created: task_01jwgsc9v5ex5vx8jf4fb5bnge
30-May-2025 14:26:23 Job status: preprocessing
30-May-2025 14:26:28 Job status: preprocessing
30-May-2025 14:26:34 Job status: running
30-May-2025 14:26:39 Job status: proce

In [19]:
HTML(f"""
<video controls>
  <source src="{generated_video}" type="video/mp4">
</video>
""")

In [20]:
prompt = "Drone view of waves crashing against the rugged cliffs along Big Sur’s garay point beach. The crashing blue waters create white-tipped waves, while the golden light of the setting sun illuminates the rocky shore. A small island with a lighthouse sits in the distance, and green shrubbery covers the cliff’s edge. The steep drop from the road down to the beach is a dramatic feat, with the cliff’s edges jutting out over the sea. This is a view that captures the raw beauty of the coast and the rugged landscape of the Pacific Coast Highway."

generated_video = sora(prompt, width=480, height=480, n_seconds=20, n_variants=1)

30-May-2025 14:26:47 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwgsd6znf4abbaaq81jdd33e', 'status': 'queued', 'created_at': 1748615207, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Drone view of waves crashing against the rugged cliffs along Big Sur’s garay point beach. The crashing blue waters create white-tipped waves, while the golden light of the setting sun illuminates the rocky shore. A small island with a lighthouse sits in the distance, and green shrubbery covers the cliff’s edge. The steep drop from the road down to the beach is a dramatic feat, with the cliff’s edges jutting out over the sea. This is a view that captures the raw beauty of the coast and the rugged landscape of the Pacific Coast Highway.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 20, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 14:26:47 Job created: task_01jwgsd6znf4abbaaq81jdd33e
30-May-2025 14:26:53 Job status: preprocessing
30-Ma

In [21]:
HTML(f"""
<video controls>
  <source src="{generated_video}" type="video/mp4">
</video>
""")

In [22]:
prompt = "An extreme close-up of an gray-haired man with a beard in his 60s, he is deep in thought pondering the history of the universe as he sits at a cafe in Paris, his eyes focus on people offscreen as they walk as he sits mostly motionless, he is dressed in a wool coat suit coat with a button-down shirt , he wears a brown beret and glasses and has a very professorial appearance, and the end he offers a subtle closed-mouth smile as if he found the answer to the mystery of life, the lighting is very cinematic with the golden light and the Parisian streets and city in the background, depth of field, cinematic 35mm film."

generated_video = sora(prompt, width=480, height=480, n_seconds=10, n_variants=1)

30-May-2025 14:27:28 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01jwgsee3gefmar8vsdcdje3z9', 'status': 'queued', 'created_at': 1748615247, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'An extreme close-up of an gray-haired man with a beard in his 60s, he is deep in thought pondering the history of the universe as he sits at a cafe in Paris, his eyes focus on people offscreen as they walk as he sits mostly motionless, he is dressed in a wool coat suit coat with a button-down shirt , he wears a brown beret and glasses and has a very professorial appearance, and the end he offers a subtle closed-mouth smile as if he found the answer to the mystery of life, the lighting is very cinematic with the golden light and the Parisian streets and city in the background, depth of field, cinematic 35mm film.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 10, 'height': 480, 'width': 480, 'failure_reason': None}

30-May-2025 14:27:28 Job created: task_

In [23]:
HTML(f"""
<video controls>
  <source src="{generated_video}" type="video/mp4">
</video>
""")

## All SORA videos

In [24]:
!ls $SORA_DIR -lh

total 40M
-rwxrwxrwx 1 root root 6.6M May 30 14:22 sora_30May2025_142226.mp4
-rwxrwxrwx 1 root root 2.1M May 30 14:24 sora_30May2025_142430.mp4
-rwxrwxrwx 1 root root 4.8M May 30 14:25 sora_30May2025_142449.mp4
-rwxrwxrwx 1 root root 2.0M May 30 14:25 sora_30May2025_142521.mp4
-rwxrwxrwx 1 root root 7.5M May 30 14:26 sora_30May2025_142539.mp4
-rwxrwxrwx 1 root root 1.9M May 30 14:26 sora_30May2025_142617.mp4
-rwxrwxrwx 1 root root  11M May 30 14:27 sora_30May2025_142647.mp4
-rwxrwxrwx 1 root root 4.2M May 30 14:27 sora_30May2025_142727.mp4


In [25]:
sora_video_files = [os.path.join(SORA_DIR, f)
               for f in sorted(os.listdir(SORA_DIR))
               if f.lower().endswith(('.mp4'))]

clips = [VideoFileClip(f) for f in sora_video_files]
final_clip = concatenate_videoclips(clips, method="compose")

final_clip.write_videofile("all_sora_videos.mp4", codec="libx264", audio_codec="aac")

{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [480, 480], 'bitrate': 5523, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc60.31.102 libx264'}}], 'input_number': 0}], 'duration': 10.0, 'bitrate': 5532, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [480, 480], 'video_bitrate': 5523, 'video_fps': 30.0, 'video_duration': 10.0, 'video_n_frames': 300}
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/imageio_ffmpeg/binaries/ffmpeg-linux-x86_64-v7.0.2 -i videos/sora_30May2025_142226.mp4 -loglevel error -f imag

MoviePy - Done !
MoviePy - video ready all_sora_videos.mp4


In [26]:
HTML(f"""
<video controls>
  <source src="all_sora_videos.mp4" type="video/mp4">
</video>
""")